In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
import seaborn as sns

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

# GM

In [3]:
#you can also query like this
blaze_gm_data = pd.read_sql_query("""
with base as
(
   select
      *,
      last_day(week_start) year_month,
      retail_value / quantity_sold per_unit_price,
      cogs / quantity_sold per_unit_cost,
      sales.retail_value - sales.total_discount + sales.total_tax sales_post_tax_post_discount,
      sales.retail_value - sales.total_discount sales_pre_tax_post_discount,
      sales.retail_value sales_pre_tax_pre_discount,
   from
      blaze_data.sales sales
      -- EXCLUDE SAMPLES,DISPLAY,PROMO and test
   where
      lower(product_category) not like '%sample%'
      and lower(product_category) not like '%display%'
      and lower(product_category) not like '%promo%'
      and lower(product_category) not like '%test%'
    and state = 'MA'
)
,
monthly_location_rev_and_cogs as
(
   select
      year_month,
      shop_id shopid,
      company_id companyid,
      sum(sales_post_tax_post_discount) rev_post_tax_post_discount,
      sum(sales_pre_tax_post_discount) rev_pre_tax_post_discount,
      sum(sales_pre_tax_pre_discount) rev_pre_tax_pre_discount,
      sum(cogs) cogs
   from
      base
      -- EXCLUDE low COGS items
   where
      cogs > 1
      -- EXCLUDE high cost, high price items
      and per_unit_price <= 500
      and per_unit_cost <= 500
   group by
      2,
      3,
      1
   order by
      2,
      3,
      1
)
,
monthly_location_rev_and_cogs_and_profit as
(
   select
      year_month,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      rev_post_tax_post_discount - cogs profit_post_tax_post_discount,
      rev_pre_tax_post_discount - cogs profit_pre_tax_post_discount,
      rev_pre_tax_pre_discount - cogs profit_pre_tax_pre_discount,
   from
      monthly_location_rev_and_cogs
)
,
monthly_location_rev_and_cogs_and_profit_and_margin as
(
   select
      year_month,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      profit_post_tax_post_discount,
      profit_pre_tax_post_discount,
      profit_pre_tax_pre_discount,
      -- margin
      profit_post_tax_post_discount / nullif(rev_post_tax_post_discount, 0) margin_post_tax_post_discount,
      profit_pre_tax_post_discount / nullif(rev_pre_tax_post_discount, 0) margin_pre_tax_post_discount,
      profit_pre_tax_pre_discount / nullif(rev_pre_tax_pre_discount, 0) margin_pre_tax_pre_discount
   from
      monthly_location_rev_and_cogs_and_profit
)
--     select * from monthly_location_rev_and_cogs_and_profit_and_margin
,
monthly_gm_analysis as
(
   select
      year_month,
      shopid,
      companyid,
      -- CHOOSE TO USE PRE TAX POST DISCOUNT
      rev_pre_tax_post_discount,
      lag(rev_pre_tax_post_discount) over (partition by shopid order by year_month) lag_rev_pre_tax_post_discount,
      cogs,
      profit_pre_tax_post_discount,
      margin_pre_tax_post_discount,
      count(distinct year_month) over (partition by shopid ) month_available,
      sum(cogs) over (partition by shopid order by year_month rows between 2 preceding and current row ) sum_cogs_3m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month rows between 1 preceding and current row ) avg_margin_2m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month rows between 2 preceding and current row ) avg_margin_3m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month rows between 5 preceding and current row ) avg_margin_6m,
      avg(margin_pre_tax_post_discount) over (partition by shopid order by year_month ) avg_margin_all
   from
      monthly_location_rev_and_cogs_and_profit_and_margin
)
select
   *,
   (
      rev_pre_tax_post_discount - lag_rev_pre_tax_post_discount
   )
   / nullif(lag_rev_pre_tax_post_discount, 0) rev_change
from
   monthly_gm_analysis
   where shopid not in ("5e50449d6c0a910843356fa3",
                        "5e666af1aa809b085a98a646",
                        "56cf846ee38179985229e59e",
                        "5e6d0948b987950831b5092c",
                        "5ab806d6c182bb072e0b0e01",
                        "617b7da0e5cb65322b32781a",
                        "56cf84e4e38179985229e59f",
                        "5ee7c6a938fa9a08c8b7939c",
                        "5eebecd661ea0b08cdbcc386",
                        "56cf855ce38179985229e5a0",
                        "5ef12ad7bb5e7a08c5b3f4f3",
                        "5f1108355ae12008f4fef76e",
                        "5f123a5d12497908d2888dde",
                        "5f2309d579373608c93e6742",
                        "5f341ed02fdd5608e0821b45",
                        "5f8870200260fe08fe708db5",
                        "5f8870200260fe08fe708db6",
                        "5fac6a0b0643cc08efd1d327",
                        "56cf855ce38179985229e5a1",
                        "5d238c1955be2e07f5ee66e5",
                        "57b35a5d920f401e40146301",
                        "603d1a9eb9d7dc08cca27203",
                        "603d1a9eb9d7dc08cca27204",
                        "6046544f61e17808cff96fdb",
                        "6046544f61e17808cff96fdc",
                        "57b35aa8920f401e40146334",
                        "57b6b145920f4045cbe5fe81",
                        "6059219abe32aa08fa3a11dc",
                        "57b6b36a920f4046337f2618",
                        "58a7433f819d9e6ebd427617",
                        "60c0ffd6331cab296f02f49a",
                        "610c0a0d381ba0747c981ce9",
                        "611c0c98127cdd740cff94db",
                        "5a43f72b819d9e3ef9373675",
                        "611c0cc403d2e97dc2c159b0",
                        "5e6d0ac5b987950831b513e3",
                        "5a4563f1819d9e4ae6207e06",
                        "611c0ce7649819050ed4906b",
                        "611c0d02f572ec7882d64eea",
                        "611c0d26d5f1e86542fe3b8b",
                        "61672305b5e382719c7e4c2a",
                        "616dee0e4497e822dae13b21",
                        "616e5b13dd10fe4d3f34fd05",
                        "616e5b13dd10fe4d3f34fd06",
                        "619834e921c8017e23135674",
                        "5ab2d1a5819d9e21137cec3c",
                        "61bb746b82fddc50e058d711",
                        "61c3be757511d81170a29d51",
                        "61d49080eec1b51a3e33dc39",
                        "61d8912bdd71d40c2b68c5b3",
                        "61ba4efb9536472be6a8baa9",
                        "61d895fb3efbfb6da5210ae3",
                        "61d8aaf527205210191d7a1e",
                        "61e72d51073c4a79cfeccb62",
                        "61fdab9eae59d05c49422c3a",
                        "6202bd1c15425361f688aabf",
                        "6202bd1c15425361f688aac0",
                        "5b9f8e1db93307080d35b85f",
                        "5c2c250eabe12808989ac2f7",
                        "58be5603c182bb1f9bcd2ad5",
                        "5c8be64665a6a40808118621",
                        "606761051436a608e166db19",
                        "5ce2eb562d4dcf07fb7d0c5f",
                        "6176fa9eda707b1faba5f717",
                        "5b517867485da70830fcc04e",
                        "5b9f841a17b6e00812fe71f3",
                        "5da023586d8ce1082b9c91bd",
                        "5bbce3db420c21083300e410",
                        "5d238c1955be2e07f5ee66e4",
                        "5b209459062bd807b142a1f6",
                        "5cb1a5839fbb5f0818f6de55",
                        "5cda1adb183d6a0801347027",
                        "5be3903b79015e083194aa70",
                        "5d1ba174cdbd6d0803c99cfd",
                        "5a8a6eb8819d9e78641ab6ec",
                        "5db863825a04a2083b464083",
                        "5d8a7fb3a3274308311743d5",
                        "5d8a7fb3a3274308311743d6",
                        "5d9e425d52d15b0824a8a5d5",
                        "5dd6da52ce4015084c72e8db",
                        "5e07f51837cdd7082662f3c6",
                        "5e4d849981e3cd08326b6c14",
                        "5dfd2308fa462a086a8bfab8",
                        "5e0ee2710a7b7008346b06b4",
                        "5e4d849981e3cd08326b6c15",
                        "5dfd2308fa462a086a8bfab9",
                        "5e6d0adc12f2e208728f9369",
                        "5e6d0948b987950831b5092b",
                        "5ed589d96c0bf408dcc92d97",
                        "5eebecd761ea0b08cdbcc387",
                        "5ef12ad7bb5e7a08c5b3f4f2",
                        "5f63afd430bd7208ce1ab304",
                        "5f63afd430bd7208ce1ab305",
                        "5bf5ce54062bd807e436b473",
                        "5fac691a715f3808cd0aa82d",
                        "5d9d7fc27e2e46081e7e69f3",
                        "6034287d52503608c0b3ca48",
                        "5dfd40ea521ea90860c91848",
                        "6034287d52503608c0b3ca49",
                        "60636227ec491108d1c970d5",
                        "5a9a2274c182bb4ec00a72ca",
                        "60aea3bb1940943cecec2dbf",
                        "60a2f1610f75543c8e7dde50",
                        "5eb98f2d649a6508f87a6436",
                        "5e62a68447d90a086b8231c0",
                        "5e62a68447d90a086b8231c1",
                        "615b634cac08aa0a794158c4",
                        "60a2db8d0f75543c8e7d513b",
                        "60a2db8d0f75543c8e7d513c",
                        "60a4051e67406e7edad8ab88",
                        "60c0ff66082dcc61e42ad4a6",
                        "60c0ff66082dcc61e42ad4a7",
                        "6046c57057abe808d1f260fd",
                        "60636227ec491108d1c970d4",
                        "610abcaec3d09859f9b630d1",
                        "606cfaadba253e08c4189a63",
                        "607a321441cfa308f6e65b58",
                        "6197e31b21c8017e2310c10e",
                        "611d457db7c6f80914292394",
                        "610c0a0d381ba0747c981cea",
                        "610abcaec3d09859f9b630d2",
                        "611c0cb3a29e0f598f108a60",
                        "5fff453a81a63608d3a1a5b7",
                        "615237cc0937e52684b6c134",
                        "620b59c97852ca7a7dcc6f05",
                        "6196aa4deb62c87eba4014ad",
                        "619816781f88a1280f8a0c28",
                        "61d7b217a42c2f124d9b76b1",
                        "61cf4b452bc6657512b92d2a",
                        "61ebb060b04dc7510465f774",
                        "61eedaa0b882812f2f46a23d",
                        "61eedade6716dd2a2e0cfce5",
                        "5e6d0ae6e4a25a0834047467",
                        "5dc3315fe0faca07c6dfe849",
                        "5f7e17594445b008d2f63ebd",
                        "5f7f63053dbcad08cc9edbfb",
                        "5fff453a81a63608d3a1a5b8",
                        "603d5425a9ca7208f886d156",
                        "60e48dbbd5596d7c8a1d24dc",
                        "611c0d335901162ff90ef4dd",
                        "613f771aa96c863c00c1517d",
                        "6153b70fd343dc6230c4b67d",
                        "61699cdf032e1c1f920ec70f",
                        "61699cdf032e1c1f920ec710",
                        "61a7b4c6c8c7d85ea1846f14",
                        "61c390cfb863cb3baddb70f4",
                        "61d7b252ae999342ac05499e",
                        "61e6f857c5da34762f1c1fb5",
                        "61ebb00a335a531542342a32",
                        "620b59c97852ca7a7dcc6f04",
                        "6226640755810d3311e8ecd0")
""",engine)

In [4]:
blaze_gm_data['year_month'] = pd.to_datetime(blaze_gm_data['year_month'])

In [5]:
blaze_gm_data

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change
0,2021-12-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,164107.25,NaN,76542.23,87565.02,0.533584,10,76542.23,0.533584,0.533584,0.533584,0.533584,NaN
1,2022-01-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,287410.71,164107.25,129108.09,158302.62,0.550789,10,205650.32,0.542186,0.542186,0.542186,0.542186,0.751359
2,2022-02-28,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,307007.31,287410.71,140429.42,166577.89,0.542586,10,346079.74,0.546687,0.542320,0.542320,0.542320,0.068183
3,2022-03-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,351007.09,307007.31,168963.00,182044.09,0.518634,10,438500.51,0.530610,0.537336,0.536398,0.536398,0.143318
4,2022-04-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,362430.54,351007.09,175306.13,187124.41,0.516304,10,484698.55,0.517469,0.525841,0.532379,0.532379,0.032545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,2022-07-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,206697.01,182869.43,117272.27,89424.74,0.432637,10,315053.54,0.428451,0.408608,0.417482,0.423645,0.130298
119,2022-08-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,258814.62,206697.01,144532.10,114282.52,0.441561,10,367088.53,0.437099,0.432821,0.421441,0.425636,0.252145
120,2022-09-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,345868.26,258814.62,175702.28,170165.98,0.491997,10,437506.65,0.466779,0.455398,0.428353,0.432272,0.336355
121,2022-02-28,61cb592938b1e14beb10bb7a,61cb592938b1e14beb10bb61,1211.00,NaN,584.68,626.32,0.517192,2,584.68,0.517192,0.517192,0.517192,0.517192,NaN


In [6]:
blaze_gm_data[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2021-08-31,6
2021-09-30,5
2021-10-31,6
2021-11-30,6
2021-12-31,8
2022-01-31,8
2022-02-28,10
2022-03-31,8
2022-04-30,10


In [7]:
blaze_gm_data[blaze_gm_data['year_month'] == '2022-07-31'][['shopid','month_available']].groupby(['shopid']).mean()

,month_available
shopid,
5e7e9d34f795d008e7136fe1,14.0
5faf10dda4594c08c922b618,14.0
5fb5656cd5322c09020a0027,14.0
5fdce3b36c42c608cc9c8b8b,14.0
602c09ee9e113008ed666e63,14.0
607701db4683fe08ef9f2b44,10.0
609d85d3b6a049435d4faf02,10.0
60b913136fc90078bb683ee1,12.0
610daf318c9aff4915efe183,7.0


In [8]:
blaze_gm_data_long = blaze_gm_data[blaze_gm_data['month_available'] >= 5]

In [9]:
blaze_gm_data_long['shopid'].nunique()

10

# 1m 

In [10]:
blaze_gm_data_long

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change
0,2021-12-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,164107.25,NaN,76542.23,87565.02,0.533584,10,76542.23,0.533584,0.533584,0.533584,0.533584,NaN
1,2022-01-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,287410.71,164107.25,129108.09,158302.62,0.550789,10,205650.32,0.542186,0.542186,0.542186,0.542186,0.751359
2,2022-02-28,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,307007.31,287410.71,140429.42,166577.89,0.542586,10,346079.74,0.546687,0.542320,0.542320,0.542320,0.068183
3,2022-03-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,351007.09,307007.31,168963.00,182044.09,0.518634,10,438500.51,0.530610,0.537336,0.536398,0.536398,0.143318
4,2022-04-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,362430.54,351007.09,175306.13,187124.41,0.516304,10,484698.55,0.517469,0.525841,0.532379,0.532379,0.032545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,2022-05-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,146569.62,112075.05,92497.11,54072.51,0.368920,10,220047.24,0.389830,0.410062,0.422043,0.422043,0.307781
117,2022-06-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,182869.43,146569.62,105284.16,77585.27,0.424266,10,263822.68,0.396593,0.401309,0.409421,0.422361,0.247663
118,2022-07-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,206697.01,182869.43,117272.27,89424.74,0.432637,10,315053.54,0.428451,0.408608,0.417482,0.423645,0.130298
119,2022-08-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,258814.62,206697.01,144532.10,114282.52,0.441561,10,367088.53,0.437099,0.432821,0.421441,0.425636,0.252145


In [11]:
# mom margin
gm_distr_mom = blaze_gm_data_long.groupby(['year_month'])['margin_pre_tax_post_discount'].describe().reset_index()
gm_distr_mom.index = pd.to_datetime(gm_distr_mom['year_month'])
gm_distr_mom

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.497434,0.011709,0.487522,0.488357,0.492450,0.504145,0.514694
2021-10-31,2021-10-31,6.0,0.498306,0.022464,0.460658,0.492747,0.499911,0.508064,0.528213
2021-11-30,2021-11-30,6.0,0.496347,0.025522,0.466641,0.479345,0.496106,0.504012,0.538823
2021-12-31,2021-12-31,8.0,0.502336,0.026345,0.473160,0.483350,0.494554,0.522181,0.542658
2022-01-31,2022-01-31,8.0,0.495824,0.052370,0.384271,0.482802,0.498250,0.530127,0.550789
2022-02-28,2022-02-28,9.0,0.527543,0.106037,0.417804,0.483056,0.487122,0.535330,0.791667
2022-03-31,2022-03-31,8.0,0.503760,0.030987,0.450525,0.486485,0.501254,0.524124,0.544857
2022-04-30,2022-04-30,10.0,0.364993,0.334167,-0.537864,0.427829,0.498626,0.512696,0.553917


In [12]:
round(gm_distr_mom[['25%','50%','75%']].mean(),2)

25%    0.49
50%    0.50
75%    0.52
dtype: float64

In [13]:
round(blaze_gm_data_long['margin_pre_tax_post_discount'].quantile(0.95),2)

0.56

# 2m

In [14]:
gm_distr_2m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_2m'].describe().reset_index()
gm_distr_2m.index = pd.to_datetime(gm_distr_2m['year_month'])
gm_distr_2m

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.500007,0.012295,0.487903,0.492865,0.498718,0.500410,0.520139
2021-10-31,2021-10-31,6.0,0.498491,0.015407,0.474090,0.494788,0.498411,0.503353,0.521454
2021-11-30,2021-11-30,6.0,0.497326,0.023375,0.463649,0.486774,0.498338,0.504981,0.533518
2021-12-31,2021-12-31,8.0,0.501897,0.025005,0.469901,0.483466,0.498684,0.514974,0.540741
2022-01-31,2022-01-31,8.0,0.499080,0.034802,0.442136,0.480756,0.491239,0.526976,0.543436
2022-02-28,2022-02-28,9.0,0.528119,0.107846,0.401038,0.482763,0.497128,0.539772,0.791667
2022-03-31,2022-03-31,8.0,0.499144,0.035093,0.434164,0.485365,0.493171,0.531387,0.540094
2022-04-30,2022-04-30,10.0,0.433018,0.149375,0.126902,0.443671,0.495790,0.513722,0.549387


In [15]:
round(gm_distr_2m[['25%','50%','75%']].mean(),2)

25%    0.48
50%    0.50
75%    0.52
dtype: float64

In [16]:
round(blaze_gm_data_long['avg_margin_2m'].quantile(0.95),2)

0.56

## 3m

In [17]:
gm_distr_3m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_3m'].describe().reset_index()
gm_distr_3m.index = pd.to_datetime(gm_distr_3m['year_month'])
gm_distr_3m

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.500007,0.012295,0.487903,0.492865,0.498718,0.500410,0.520139
2021-10-31,2021-10-31,6.0,0.499983,0.013427,0.486031,0.490830,0.497948,0.504497,0.522830
2021-11-30,2021-11-30,6.0,0.497797,0.018548,0.471607,0.489641,0.497643,0.503665,0.527243
2021-12-31,2021-12-31,8.0,0.502241,0.023884,0.466820,0.486560,0.499938,0.515062,0.536565
2022-01-31,2022-01-31,8.0,0.497820,0.033892,0.442136,0.481977,0.490894,0.521218,0.542186
2022-02-28,2022-02-28,9.0,0.530241,0.103802,0.434025,0.482916,0.493352,0.540734,0.791667
2022-03-31,2022-03-31,8.0,0.498037,0.040410,0.417533,0.486161,0.493715,0.532551,0.541467
2022-04-30,2022-04-30,10.0,0.431072,0.148508,0.126902,0.439959,0.492839,0.518382,0.544701


In [18]:
round(gm_distr_3m[['25%','50%','75%']].mean(),2)

25%    0.48
50%    0.50
75%    0.52
dtype: float64

In [19]:
round(blaze_gm_data_long['avg_margin_3m'].quantile(0.95),2)

0.55

## 6m

In [20]:
gm_distr_6m = blaze_gm_data_long.groupby(['year_month'])['avg_margin_6m'].describe().reset_index()
gm_distr_6m.index = pd.to_datetime(gm_distr_6m['year_month'])
gm_distr_6m

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.502580,0.019307,0.481585,0.483356,0.509915,0.512462,0.525584
2021-09-30,2021-09-30,5.0,0.500007,0.012295,0.487903,0.492865,0.498718,0.500410,0.520139
2021-10-31,2021-10-31,6.0,0.499983,0.013427,0.486031,0.490830,0.497948,0.504497,0.522830
2021-11-30,2021-11-30,6.0,0.499104,0.016375,0.481184,0.487959,0.497488,0.504514,0.526828
2021-12-31,2021-12-31,8.0,0.503019,0.019653,0.479579,0.488586,0.499938,0.511621,0.533584
2022-01-31,2022-01-31,8.0,0.497538,0.031216,0.442136,0.486461,0.492688,0.514705,0.542186
2022-02-28,2022-02-28,9.0,0.529027,0.103511,0.434025,0.488457,0.491853,0.533989,0.791667
2022-03-31,2022-03-31,8.0,0.497679,0.033461,0.438150,0.485566,0.491203,0.524041,0.539016
2022-04-30,2022-04-30,10.0,0.430122,0.147573,0.126902,0.445262,0.488291,0.516945,0.543300


In [21]:
round(gm_distr_6m[['25%','50%','75%']].mean(),2)

25%    0.47
50%    0.50
75%    0.52
dtype: float64

In [22]:
round(blaze_gm_data_long['avg_margin_6m'].quantile(0.95),2)

0.54

## all time

In [23]:
gm_distr_all = blaze_gm_data_long.groupby(['year_month'])['avg_margin_all'].describe().reset_index()
gm_distr_all.index = pd.to_datetime(gm_distr_all['year_month'])
round(gm_distr_all,3)

,year_month,count,mean,std,min,25%,50%,75%,max
year_month,,,,,,,,,
2021-08-31,2021-08-31,5.0,0.503,0.019,0.482,0.483,0.510,0.512,0.526
2021-09-30,2021-09-30,5.0,0.500,0.012,0.488,0.493,0.499,0.500,0.520
2021-10-31,2021-10-31,6.0,0.500,0.013,0.486,0.491,0.498,0.504,0.523
2021-11-30,2021-11-30,6.0,0.499,0.016,0.481,0.488,0.497,0.505,0.527
2021-12-31,2021-12-31,8.0,0.503,0.020,0.480,0.489,0.500,0.512,0.534
2022-01-31,2022-01-31,8.0,0.498,0.031,0.442,0.486,0.493,0.515,0.542
2022-02-28,2022-02-28,9.0,0.529,0.103,0.434,0.487,0.492,0.533,0.792
2022-03-31,2022-03-31,8.0,0.497,0.032,0.438,0.488,0.491,0.520,0.536
2022-04-30,2022-04-30,10.0,0.429,0.147,0.127,0.446,0.490,0.511,0.536


In [24]:
round(gm_distr_all[['25%','50%','75%']].mean(),2)

25%    0.47
50%    0.50
75%    0.51
dtype: float64

In [25]:
round(blaze_gm_data_long['avg_margin_all'].quantile(0.95),2)

0.54

# Rev change

In [26]:
def gmv_change_variance_point_mapping(x):
    if x < -0.1:
        return -5
    elif x >= -0.1 and x <-0.05:
        return -2.5
    elif x >= -0.05 and x < 0:
        return 0
    elif x >= 0 and x < 0.05:
        return 5
    elif x >= 0.05 and x <= 0.5:
        return 10
    else:
        return numpy.nan
    

## blaze benchmark

In [ ]:
# rev_change_bm = blaze_gm_data_long.groupby(['year_month'])['rev_change'].describe().reset_index()
# rev_change_bm.index = pd.to_datetime(rev_change_bm['year_month'])
# rev_change_bm

In [ ]:
# rev_change_bm[['50%']].plot(kind = 'bar')

In [ ]:
# rev_change_bm[['year_month','50%']]

In [ ]:
# rev_change_bm_with_weight = rev_change_bm[['year_month','50%']]
# rev_change_bm_with_weight['weight'] = [0.36,0.44,0.52,0.6,0.68,0.76,0.84,0.92,1]
# rev_change_bm_with_weight.columns = ['year_month','rev_change_bm','rev_change_weight']
# rev_change_bm_with_weight = rev_change_bm_with_weight.reset_index(drop = True)
# rev_change_bm_with_weight

In [27]:
gm_distr_all['year_month']

year_month
2021-08-31   2021-08-31
2021-09-30   2021-09-30
2021-10-31   2021-10-31
2021-11-30   2021-11-30
2021-12-31   2021-12-31
2022-01-31   2022-01-31
2022-02-28   2022-02-28
2022-03-31   2022-03-31
2022-04-30   2022-04-30
2022-05-31   2022-05-31
2022-06-30   2022-06-30
2022-07-31   2022-07-31
2022-08-31   2022-08-31
2022-09-30   2022-09-30
Name: year_month, dtype: datetime64[ns]

## state website published benchmark

In [30]:
# use own bm

rev_change_bm_with_weight = pd.DataFrame([numpy.nan,-0.07, 0.01, -0.07, 0.09,0.07,0.02,0.11,-0.01,-0.05,-0.01,-0.07,numpy.nan,numpy.nan])
rev_change_bm_with_weight['year_month'] = gm_distr_all['year_month'].values
rev_change_bm_with_weight['weight'] = [0.12,0.2,0.28,0.36,0.44,0.52,0.6,0.68,0.76,0.84,0.92,1,numpy.nan,numpy.nan]
rev_change_bm_with_weight.columns = ['rev_change_bm','year_month','rev_change_weight']
rev_change_bm_with_weight = rev_change_bm_with_weight.reset_index(drop = True)
rev_change_bm_with_weight

,rev_change_bm,year_month,rev_change_weight
0,NaN,2021-08-31,0.12
1,-0.07,2021-09-30,0.20
2,0.01,2021-10-31,0.28
3,-0.07,2021-11-30,0.36
4,0.09,2021-12-31,0.44
5,0.07,2022-01-31,0.52
6,0.02,2022-02-28,0.60
7,0.11,2022-03-31,0.68
8,-0.01,2022-04-30,0.76
9,-0.05,2022-05-31,0.84


In [31]:

blaze_gm_data_long_with_rev  = pd.merge(blaze_gm_data_long,rev_change_bm_with_weight,on=['year_month'], how='inner')
blaze_gm_data_long_with_rev['rev_change_var'] = blaze_gm_data_long_with_rev['rev_change'] - blaze_gm_data_long_with_rev['rev_change_bm']
blaze_gm_data_long_with_rev['rev_change_points'] = [gmv_change_variance_point_mapping(n) for n in blaze_gm_data_long_with_rev['rev_change_var']]
blaze_gm_data_long_with_rev['rev_change_total'] = blaze_gm_data_long_with_rev['rev_change_points'] * blaze_gm_data_long_with_rev['rev_change_weight']



In [33]:
blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['year_month']=='2022-07-31']

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total
63,2022-07-31,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,450610.91,467480.82,214843.93,235766.98,0.523216,10,630617.99,0.518626,0.519010,0.522425,0.527366,-0.036087,-0.07,1.0,0.033913,5.0,5.0
64,2022-07-31,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,438864.07,437558.70,217095.65,221768.42,0.505324,12,652331.38,0.498735,0.493430,0.487098,0.490513,0.002983,-0.07,1.0,0.072983,10.0,10.0
65,2022-07-31,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,788297.92,753911.00,332169.08,456128.84,0.578625,14,991351.01,0.572789,0.569303,0.557002,0.544683,0.045611,-0.07,1.0,0.115611,10.0,10.0
66,2022-07-31,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,170908.63,137591.99,78794.45,92114.18,0.538967,6,161352.04,0.538823,0.534990,0.448413,0.448413,0.242141,-0.07,1.0,0.312141,10.0,10.0
67,2022-07-31,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,366358.36,366291.76,170178.35,196180.01,0.535487,14,495306.89,0.530735,0.526428,0.531109,0.519964,0.000182,-0.07,1.0,0.070182,10.0,10.0
68,2022-07-31,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,358054.58,416981.18,184933.09,173121.49,0.483506,14,596707.66,0.484972,0.487696,0.488699,0.484955,-0.141317,-0.07,1.0,-0.071317,-2.5,-2.5
69,2022-07-31,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,1043922.45,1021245.25,508012.36,535910.09,0.513362,14,1488357.10,0.508164,0.507864,0.501920,0.495831,0.022205,-0.07,1.0,0.092205,10.0,10.0
70,2022-07-31,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,156107.94,131007.66,73680.29,82427.65,0.528017,7,163293.87,0.511095,0.500216,0.350890,0.350890,0.191594,-0.07,1.0,0.261594,10.0,10.0
71,2022-07-31,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,493583.94,507291.25,242822.19,250761.75,0.508043,14,741580.68,0.508610,0.506937,0.501471,0.494841,-0.027021,-0.07,1.0,0.042979,5.0,5.0
72,2022-07-31,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,206697.01,182869.43,117272.27,89424.74,0.432637,10,315053.54,0.428451,0.408608,0.417482,0.423645,0.130298,-0.07,1.0,0.200298,10.0,10.0


# inventory

In [34]:
blaze_inv_data_all = pd.read_sql_query("""with base as (
    select *,
    DATE_DIFF(date,purchase_date,DAY) diff,
    current_quantity * unit_cost total_value
    from blaze_data.inventory
    where  lower(product_category) not like '%sample%'
    and lower(product_category) not like '%display%'
    and lower(product_category) not like '%promo%'
    and lower(product_category) not like '%test%'
    -- EXCLUDE SMALL COGS
    and unit_cost > 0.01
    and current_quantity > 0
    and current_quantity < 1000
    and date in ('2022-09-01','2022-09-03','2022-09-10','2022-09-17','2022-09-24')
    and state = 'MA'
),
base_by_group as (
select *,
case when diff < 180 then 'fresh'
     when diff >= 180 and diff < 270 then 'stale'
    else  'bad' end as inventory_group
from base
),
weekly_inv as (
    select date,
           shop_id          shopid,
           company_id       companyid,
           inventory_group,
           sum(total_value) inventory
    from base_by_group
    group by 1, 2, 3,4
)
select
shopid,
companyid,
inventory_group,
avg(inventory) inventory,
last_day('2022-09-30') year_month
from weekly_inv
group by 1,2,3""",engine)

In [35]:
blaze_inv_data_all['year_month'] = pd.to_datetime(blaze_inv_data_all['year_month'])

In [36]:
set(blaze_gm_data['shopid']) - set(blaze_inv_data_all['shopid'])

set()

In [37]:
fresh_inv = blaze_inv_data_all[blaze_inv_data_all['inventory_group'] == 'fresh'][['year_month','shopid','companyid','inventory']]
stale_inv = blaze_inv_data_all[blaze_inv_data_all['inventory_group'] == 'stale'][['year_month','shopid','companyid','inventory']]


fresh_and_stale_inv = fresh_inv.merge(stale_inv, on = ['shopid','companyid','year_month'],how = 'outer')
fresh_and_stale_inv = fresh_and_stale_inv.fillna(0)

fresh_and_stale_inv.columns = ['year_month','shopid','companyid','inventory_fresh','inventory_stale']
fresh_and_stale_inv['inventory_fresh_and_stale'] = fresh_and_stale_inv['inventory_fresh']  + fresh_and_stale_inv['inventory_stale']

In [38]:
fresh_and_stale_inv

,year_month,shopid,companyid,inventory_fresh,inventory_stale,inventory_fresh_and_stale
0,2022-09-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,99838.126000,2434.430000,102272.556000
1,2022-09-30,62aa16506f3b185f56721fb2,62a7b2c8d78cf65bd398cff2,12604.000000,0.000000,12604.000000
2,2022-09-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,178099.270333,0.000000,178099.270333
3,2022-09-30,62b4b86a23efb97aa2138b78,5faf10dda4594c08c922b600,142038.171000,6878.200000,148916.371000
4,2022-09-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,260825.138095,7339.096667,268164.234762
5,2022-09-30,62a0bd372a174407d97be49f,62a0bd372a174407d97be484,13959.514000,0.000000,13959.514000
6,2022-09-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,227860.525167,11261.612000,239122.137167
7,2022-09-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,250532.120333,30624.120000,281156.240333
8,2022-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,215927.094387,32325.320000,248252.414387
9,2022-09-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,260210.340324,12189.718000,272400.058324


In [39]:
set(blaze_gm_data['shopid']) - set(fresh_and_stale_inv['shopid'])

{'605cef20813d4c08ebf50c06'}

# joined

In [40]:
data_joined = pd.merge(blaze_gm_data_long_with_rev,fresh_and_stale_inv,on=['year_month','shopid','companyid'], how='inner')


In [41]:
data_joined

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale
0,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,448020.93,218091.65,244524.61,0.528569,10,645830.08,0.526690,0.525532,0.521119,0.527231,0.032577,NaN,NaN,NaN,NaN,NaN,296943.076000,47574.284000,344517.360000
1,2022-09-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,500332.58,447181.51,250567.88,249764.70,0.499197,12,688131.56,0.503090,0.503835,0.494262,0.492609,0.118858,NaN,NaN,NaN,NaN,NaN,279636.169667,5527.489333,285163.659000
2,2022-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,636730.75,709714.60,258661.77,378068.98,0.593766,14,884323.87,0.590115,0.586285,0.573676,0.551173,-0.102835,NaN,NaN,NaN,NaN,NaN,215927.094387,32325.320000,248252.414387
3,2022-09-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,218596.06,191325.54,97528.40,121067.66,0.553842,6,261667.02,0.553887,0.548914,0.483571,0.483571,0.142535,NaN,NaN,NaN,NaN,NaN,178099.270333,0.000000,178099.270333
4,2022-09-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,366812.95,366086.20,167578.21,199234.74,0.543151,14,503396.02,0.545345,0.542059,0.534984,0.523590,0.001985,NaN,NaN,NaN,NaN,NaN,227860.525167,11261.612000,239122.137167
5,2022-09-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,365122.15,358998.01,178329.67,186792.48,0.511589,14,535724.43,0.515596,0.504899,0.498357,0.489332,0.017059,NaN,NaN,NaN,NaN,NaN,260210.340324,12189.718000,272400.058324
6,2022-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,1043956.68,1075289.27,492541.26,551415.42,0.528198,14,1505519.11,0.529294,0.523984,0.514009,0.500611,-0.029139,NaN,NaN,NaN,NaN,NaN,360785.283333,2373.016000,363158.299333
7,2022-09-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,160258.92,156107.56,74694.29,85564.63,0.533915,7,219712.83,0.538467,0.534984,0.339953,0.404484,0.026593,NaN,NaN,NaN,NaN,NaN,99838.126000,2434.430000,102272.556000
8,2022-09-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,508372.13,490704.93,258151.46,250220.67,0.492200,14,746144.68,0.496285,0.500204,0.502461,0.495048,0.036004,NaN,NaN,NaN,NaN,NaN,260825.138095,7339.096667,268164.234762
9,2022-09-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,345868.26,258814.62,175702.28,170165.98,0.491997,10,437506.65,0.466779,0.455398,0.428353,0.432272,0.336355,NaN,NaN,NaN,NaN,NaN,250532.120333,30624.120000,281156.240333


In [42]:
data_joined['inventory_turnover'] = data_joined['sum_cogs_3m'] / data_joined['inventory_fresh_and_stale'] * 4
data_joined['year_month'] = pd.to_datetime(data_joined['year_month'])

In [43]:
data_joined[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2022-09-30,10


In [44]:
data_joined[data_joined['year_month'] >= '2021-10-31'].groupby(['year_month'])['inventory_turnover'].describe().reset_index()

,year_month,count,mean,std,min,25%,50%,75%,max
0,2022-09-30,10.0,9.609375,3.460733,5.87688,7.590464,8.506981,10.760361,16.582511


In [45]:
round(data_joined['inventory_turnover'].quantile(0.95),2)

15.53

In [ ]:
# data_joined_week = pd.merge(blaze_gm_data_long_with_rev,blaze_inv_data,on=['shopid','companyid'], how='inner')

# data_joined_week['inventory_turnover'] = data_joined_week['sum_cogs_3m'] / data_joined_week['inventory'] * 4
# data_joined_week['year_month'] = pd.to_datetime(data_joined_week['year_month'])

# data_joined_week

# UW

In [46]:
recent = data_joined[data_joined['year_month'] == '2022-09-30']
recent = recent.reset_index(drop = True)

In [47]:
recent[['year_month','shopid']].groupby(['year_month']).nunique()

,shopid
year_month,
2022-09-30,10


In [48]:
recent

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover
0,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,448020.93,218091.65,244524.61,0.528569,10,645830.08,0.526690,0.525532,0.521119,0.527231,0.032577,NaN,NaN,NaN,NaN,NaN,296943.076000,47574.284000,344517.360000,7.498375
1,2022-09-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,500332.58,447181.51,250567.88,249764.70,0.499197,12,688131.56,0.503090,0.503835,0.494262,0.492609,0.118858,NaN,NaN,NaN,NaN,NaN,279636.169667,5527.489333,285163.659000,9.652444
2,2022-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,636730.75,709714.60,258661.77,378068.98,0.593766,14,884323.87,0.590115,0.586285,0.573676,0.551173,-0.102835,NaN,NaN,NaN,NaN,NaN,215927.094387,32325.320000,248252.414387,14.248786
3,2022-09-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,218596.06,191325.54,97528.40,121067.66,0.553842,6,261667.02,0.553887,0.548914,0.483571,0.483571,0.142535,NaN,NaN,NaN,NaN,NaN,178099.270333,0.000000,178099.270333,5.876880
4,2022-09-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,366812.95,366086.20,167578.21,199234.74,0.543151,14,503396.02,0.545345,0.542059,0.534984,0.523590,0.001985,NaN,NaN,NaN,NaN,NaN,227860.525167,11261.612000,239122.137167,8.420735
5,2022-09-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,365122.15,358998.01,178329.67,186792.48,0.511589,14,535724.43,0.515596,0.504899,0.498357,0.489332,0.017059,NaN,NaN,NaN,NaN,NaN,260210.340324,12189.718000,272400.058324,7.866730
6,2022-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,1043956.68,1075289.27,492541.26,551415.42,0.528198,14,1505519.11,0.529294,0.523984,0.514009,0.500611,-0.029139,NaN,NaN,NaN,NaN,NaN,360785.283333,2373.016000,363158.299333,16.582511
7,2022-09-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,160258.92,156107.56,74694.29,85564.63,0.533915,7,219712.83,0.538467,0.534984,0.339953,0.404484,0.026593,NaN,NaN,NaN,NaN,NaN,99838.126000,2434.430000,102272.556000,8.593227
8,2022-09-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,508372.13,490704.93,258151.46,250220.67,0.492200,14,746144.68,0.496285,0.500204,0.502461,0.495048,0.036004,NaN,NaN,NaN,NaN,NaN,260825.138095,7339.096667,268164.234762,11.129667
9,2022-09-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,345868.26,258814.62,175702.28,170165.98,0.491997,10,437506.65,0.466779,0.455398,0.428353,0.432272,0.336355,NaN,NaN,NaN,NaN,NaN,250532.120333,30624.120000,281156.240333,6.224392


In [ ]:
# recent_week = data_joined_week[data_joined_week['year_month'] == '2022-04-30']
# recent_week = recent_week.reset_index(drop = True)

# recent_week[['shopid','inventory_turnover']].groupby(['shopid']).describe()

## gm % score

In [49]:
def get_gm_perc_scores(threshold,gm_3,gm_6,gm_all):
    # 3m score
    if gm_3 <= threshold[0][0]:
        score_3m = -2
    elif threshold[0][0] < gm_3 <= threshold[0][1]:
        score_3m = 0
    elif threshold[0][1] < gm_3 <= threshold[0][2]:
        score_3m = 5
    elif threshold[0][2] < gm_3 <= threshold[0][3]:
        score_3m = 5
    else:
        score_3m = 0
    # 6m score
    if gm_6 <= threshold[1][0]:
        score_6m = -2
    elif threshold[1][0] < gm_6 <= threshold[1][1]:
        score_6m = 0
    elif threshold[1][1] < gm_6 <= threshold[1][2]:
        score_6m = 5
    elif threshold[1][2] < gm_6 <= threshold[1][3]:
        score_6m = 5
    else:
        score_6m = 0
    # all time score
    if gm_all <= threshold[2][0]:
        score_all = -2
    elif threshold[2][0] < gm_all <= threshold[2][1]:
        score_all = 0
    elif threshold[2][1] < gm_all <= threshold[2][2]:
        score_all = 5
    elif threshold[2][2] < gm_all <= threshold[2][3]:
        score_all = 5
    else:
        score_all = 0
    return score_3m,score_6m,score_all
    

In [50]:
gm_threshold = [[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7],[0.49,0.50,0.51,0.7]]

In [ ]:
#gm_threshold = [[0.46,0.52,0.58,0.7],[0.46,0.52,0.58,0.7],[0.46,0.52,0.58,0.7]]

In [ ]:
#gm_threshold = [[0.45,0.51,0.57,0.7],[0.45,0.51,0.57,0.7],[0.45,0.51,0.57,0.7]]

In [ ]:
#gm_threshold = [[0.44,0.50,0.56,0.7],[0.44,0.50,0.56,0.7],[0.44,0.50,0.56,0.7]]

In [51]:
# use 3,6,9m
gm_score_data = recent.apply(lambda row: get_gm_perc_scores(gm_threshold,row['avg_margin_3m'],row['avg_margin_6m'],row['avg_margin_all']),axis = 1)
recent_with_gm = pd.concat([recent, pd.DataFrame([list(y) for y in gm_score_data.values], columns = ['gm_3m_score', 'gm_6m_score','gm_all_score'])],axis=1)



In [ ]:
# # use 1,2,3m
# gm_score_data = recent.apply(lambda row: get_gm_perc_scores(gm_threshold,row['margin_pre_tax_post_discount'],row['avg_margin_2m'],row['avg_margin_3m']),axis = 1)
# recent_with_gm = pd.concat([recent, pd.DataFrame([list(y) for y in gm_score_data.values], columns = ['gm_1m_score', 'gm_2m_score','gm_3m_score'])],axis=1)



In [ ]:
recent_with_gm

## inventory turnover score

In [52]:
recent_with_gm['turnover_score'] = [10 if (recent_with_gm['inventory_turnover'][i] >= 6 and recent_with_gm['inventory_turnover'][i] <= 26) else 0 for i in range(len(recent_with_gm))]



In [53]:
recent_with_gm

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score
0,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,448020.93,218091.65,244524.61,0.528569,10,645830.08,0.526690,0.525532,0.521119,0.527231,0.032577,NaN,NaN,NaN,NaN,NaN,296943.076000,47574.284000,344517.360000,7.498375,5,5,5,10
1,2022-09-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,500332.58,447181.51,250567.88,249764.70,0.499197,12,688131.56,0.503090,0.503835,0.494262,0.492609,0.118858,NaN,NaN,NaN,NaN,NaN,279636.169667,5527.489333,285163.659000,9.652444,5,0,0,10
2,2022-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,636730.75,709714.60,258661.77,378068.98,0.593766,14,884323.87,0.590115,0.586285,0.573676,0.551173,-0.102835,NaN,NaN,NaN,NaN,NaN,215927.094387,32325.320000,248252.414387,14.248786,5,5,5,10
3,2022-09-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,218596.06,191325.54,97528.40,121067.66,0.553842,6,261667.02,0.553887,0.548914,0.483571,0.483571,0.142535,NaN,NaN,NaN,NaN,NaN,178099.270333,0.000000,178099.270333,5.876880,5,-2,-2,0
4,2022-09-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,366812.95,366086.20,167578.21,199234.74,0.543151,14,503396.02,0.545345,0.542059,0.534984,0.523590,0.001985,NaN,NaN,NaN,NaN,NaN,227860.525167,11261.612000,239122.137167,8.420735,5,5,5,10
5,2022-09-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,365122.15,358998.01,178329.67,186792.48,0.511589,14,535724.43,0.515596,0.504899,0.498357,0.489332,0.017059,NaN,NaN,NaN,NaN,NaN,260210.340324,12189.718000,272400.058324,7.866730,5,0,-2,10
6,2022-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,1043956.68,1075289.27,492541.26,551415.42,0.528198,14,1505519.11,0.529294,0.523984,0.514009,0.500611,-0.029139,NaN,NaN,NaN,NaN,NaN,360785.283333,2373.016000,363158.299333,16.582511,5,5,5,10
7,2022-09-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,160258.92,156107.56,74694.29,85564.63,0.533915,7,219712.83,0.538467,0.534984,0.339953,0.404484,0.026593,NaN,NaN,NaN,NaN,NaN,99838.126000,2434.430000,102272.556000,8.593227,5,-2,-2,10
8,2022-09-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,508372.13,490704.93,258151.46,250220.67,0.492200,14,746144.68,0.496285,0.500204,0.502461,0.495048,0.036004,NaN,NaN,NaN,NaN,NaN,260825.138095,7339.096667,268164.234762,11.129667,5,5,0,10
9,2022-09-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,345868.26,258814.62,175702.28,170165.98,0.491997,10,437506.65,0.466779,0.455398,0.428353,0.432272,0.336355,NaN,NaN,NaN,NaN,NaN,250532.120333,30624.120000,281156.240333,6.224392,-2,-2,-2,10


## rev change score

In [ ]:
#blaze_gm_data_long_with_rev[blaze_gm_data_long_with_rev['shopid'] == '60b913136fc90078bb683ee1']

In [54]:
rev_change_score = blaze_gm_data_long_with_rev[['shopid','rev_change_total']].groupby('shopid').sum().reset_index()

In [55]:
rev_change_score['rev_change_score'] = [max(min(round(rev_change_score['rev_change_total'][i],2),10),-10) for i in range(len(rev_change_score))]
rev_change_score.sort_values(by = 'rev_change_total')


,shopid,rev_change_total,rev_change_score
8,610daf318c9aff4915efe183,10.0,10.0
9,623e39dfbcd2bd3c1fb82f46,10.0,10.0
0,5e7e9d34f795d008e7136fe1,13.3,10.0
4,602c09ee9e113008ed666e63,18.3,10.0
3,5fdce3b36c42c608cc9c8b8b,30.8,10.0
6,609d85d3b6a049435d4faf02,31.4,10.0
5,607701db4683fe08ef9f2b44,32.8,10.0
7,60b913136fc90078bb683ee1,37.4,10.0
1,5faf10dda4594c08c922b618,45.7,10.0
2,5fb5656cd5322c09020a0027,50.1,10.0


## gm $ score

In [56]:
gm_dollar_score = blaze_gm_data_long_with_rev[['shopid','profit_pre_tax_post_discount']].groupby('shopid').mean().reset_index()

In [57]:
gm_dollar_score['gm_dollar_score'] = [15 if gm_dollar_score['profit_pre_tax_post_discount'][i] >= 200000 else 0 for i in range(len(rev_change_score))]



In [58]:
recent_with_gm_and_rev_change = pd.merge(recent_with_gm,rev_change_score[['shopid','rev_change_score']],on=['shopid'], how='inner')
recent_with_gm_and_rev_change

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score
0,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,448020.93,218091.65,244524.61,0.528569,10,645830.08,0.526690,0.525532,0.521119,0.527231,0.032577,NaN,NaN,NaN,NaN,NaN,296943.076000,47574.284000,344517.360000,7.498375,5,5,5,10,10.0
1,2022-09-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,500332.58,447181.51,250567.88,249764.70,0.499197,12,688131.56,0.503090,0.503835,0.494262,0.492609,0.118858,NaN,NaN,NaN,NaN,NaN,279636.169667,5527.489333,285163.659000,9.652444,5,0,0,10,10.0
2,2022-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,636730.75,709714.60,258661.77,378068.98,0.593766,14,884323.87,0.590115,0.586285,0.573676,0.551173,-0.102835,NaN,NaN,NaN,NaN,NaN,215927.094387,32325.320000,248252.414387,14.248786,5,5,5,10,10.0
3,2022-09-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,218596.06,191325.54,97528.40,121067.66,0.553842,6,261667.02,0.553887,0.548914,0.483571,0.483571,0.142535,NaN,NaN,NaN,NaN,NaN,178099.270333,0.000000,178099.270333,5.876880,5,-2,-2,0,10.0
4,2022-09-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,366812.95,366086.20,167578.21,199234.74,0.543151,14,503396.02,0.545345,0.542059,0.534984,0.523590,0.001985,NaN,NaN,NaN,NaN,NaN,227860.525167,11261.612000,239122.137167,8.420735,5,5,5,10,10.0
5,2022-09-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,365122.15,358998.01,178329.67,186792.48,0.511589,14,535724.43,0.515596,0.504899,0.498357,0.489332,0.017059,NaN,NaN,NaN,NaN,NaN,260210.340324,12189.718000,272400.058324,7.866730,5,0,-2,10,10.0
6,2022-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,1043956.68,1075289.27,492541.26,551415.42,0.528198,14,1505519.11,0.529294,0.523984,0.514009,0.500611,-0.029139,NaN,NaN,NaN,NaN,NaN,360785.283333,2373.016000,363158.299333,16.582511,5,5,5,10,10.0
7,2022-09-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,160258.92,156107.56,74694.29,85564.63,0.533915,7,219712.83,0.538467,0.534984,0.339953,0.404484,0.026593,NaN,NaN,NaN,NaN,NaN,99838.126000,2434.430000,102272.556000,8.593227,5,-2,-2,10,10.0
8,2022-09-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,508372.13,490704.93,258151.46,250220.67,0.492200,14,746144.68,0.496285,0.500204,0.502461,0.495048,0.036004,NaN,NaN,NaN,NaN,NaN,260825.138095,7339.096667,268164.234762,11.129667,5,5,0,10,10.0
9,2022-09-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,345868.26,258814.62,175702.28,170165.98,0.491997,10,437506.65,0.466779,0.455398,0.428353,0.432272,0.336355,NaN,NaN,NaN,NaN,NaN,250532.120333,30624.120000,281156.240333,6.224392,-2,-2,-2,10,10.0


In [59]:
final = pd.merge(recent_with_gm_and_rev_change,gm_dollar_score[['shopid','gm_dollar_score']],on=['shopid'], how='inner')



In [62]:
final['total_gm_perc_score'] = final['gm_3m_score'] + final['gm_6m_score'] + final['gm_all_score']
final['total_gm_score'] = [min(final['gm_dollar_score'][i]+final['total_gm_perc_score'][i],15) for i in range(len(final))]
final['total_score'] = final['total_gm_score'] + final['rev_change_score'] + final['turnover_score']
final['credit_limit_raw'] = [round(min(final['sum_cogs_3m'][i],final['inventory_fresh'][i]),0) for i in range(len(final))]
final['potential_credit_limit'] = round(final['credit_limit_raw'],-4) 
final['initial_credit_limit'] = final['potential_credit_limit'] / 2                         



In [63]:
final

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,448020.93,218091.65,244524.61,0.528569,10,645830.08,0.526690,0.525532,0.521119,0.527231,0.032577,NaN,NaN,NaN,NaN,NaN,296943.076000,47574.284000,344517.360000,7.498375,5,5,5,10,10.0,0,15,15,35.0,296943.0,300000.0,150000.0
1,2022-09-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,500332.58,447181.51,250567.88,249764.70,0.499197,12,688131.56,0.503090,0.503835,0.494262,0.492609,0.118858,NaN,NaN,NaN,NaN,NaN,279636.169667,5527.489333,285163.659000,9.652444,5,0,0,10,10.0,0,5,5,25.0,279636.0,280000.0,140000.0
2,2022-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,636730.75,709714.60,258661.77,378068.98,0.593766,14,884323.87,0.590115,0.586285,0.573676,0.551173,-0.102835,NaN,NaN,NaN,NaN,NaN,215927.094387,32325.320000,248252.414387,14.248786,5,5,5,10,10.0,15,15,15,35.0,215927.0,220000.0,110000.0
3,2022-09-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,218596.06,191325.54,97528.40,121067.66,0.553842,6,261667.02,0.553887,0.548914,0.483571,0.483571,0.142535,NaN,NaN,NaN,NaN,NaN,178099.270333,0.000000,178099.270333,5.876880,5,-2,-2,0,10.0,0,1,1,11.0,178099.0,180000.0,90000.0
4,2022-09-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,366812.95,366086.20,167578.21,199234.74,0.543151,14,503396.02,0.545345,0.542059,0.534984,0.523590,0.001985,NaN,NaN,NaN,NaN,NaN,227860.525167,11261.612000,239122.137167,8.420735,5,5,5,10,10.0,0,15,15,35.0,227861.0,230000.0,115000.0
5,2022-09-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,365122.15,358998.01,178329.67,186792.48,0.511589,14,535724.43,0.515596,0.504899,0.498357,0.489332,0.017059,NaN,NaN,NaN,NaN,NaN,260210.340324,12189.718000,272400.058324,7.866730,5,0,-2,10,10.0,0,3,3,23.0,260210.0,260000.0,130000.0
6,2022-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,1043956.68,1075289.27,492541.26,551415.42,0.528198,14,1505519.11,0.529294,0.523984,0.514009,0.500611,-0.029139,NaN,NaN,NaN,NaN,NaN,360785.283333,2373.016000,363158.299333,16.582511,5,5,5,10,10.0,15,15,15,35.0,360785.0,360000.0,180000.0
7,2022-09-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,160258.92,156107.56,74694.29,85564.63,0.533915,7,219712.83,0.538467,0.534984,0.339953,0.404484,0.026593,NaN,NaN,NaN,NaN,NaN,99838.126000,2434.430000,102272.556000,8.593227,5,-2,-2,10,10.0,0,1,1,21.0,99838.0,100000.0,50000.0
8,2022-09-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,508372.13,490704.93,258151.46,250220.67,0.492200,14,746144.68,0.496285,0.500204,0.502461,0.495048,0.036004,NaN,NaN,NaN,NaN,NaN,260825.138095,7339.096667,268164.234762,11.129667,5,5,0,10,10.0,15,10,15,35.0,260825.0,260000.0,130000.0
9,2022-09-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,345868.26,258814.62,175702.28,170165.98,0.491997,10,437506.65,0.466779,0.455398,0.428353,0.432272,0.336355,NaN,NaN,NaN,NaN,NaN,250532.120333,30624.120000,281156.240333,6.224392,-2,-2,-2,10,10.0,0,-6,-6,14.0,250532.0,250000.0,125000.0


In [64]:
final[['year_month', 'shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs', 'profit_pre_tax_post_discount',
       'margin_pre_tax_post_discount', 'sum_cogs_3m',
       'avg_margin_2m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all', 'inventory_fresh_and_stale',
       'inventory_turnover', 'gm_3m_score', 'gm_6m_score', 'gm_all_score', 'total_gm_perc_score','gm_dollar_score',
       'total_gm_score','turnover_score', 'rev_change_score',  'total_score','credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit']]


,year_month,shopid,companyid,rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,total_gm_perc_score,gm_dollar_score,total_gm_score,turnover_score,rev_change_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,218091.65,244524.61,0.528569,645830.08,0.526690,0.525532,0.521119,0.527231,344517.360000,7.498375,5,5,5,15,0,15,10,10.0,35.0,296943.0,300000.0,150000.0
1,2022-09-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,500332.58,250567.88,249764.70,0.499197,688131.56,0.503090,0.503835,0.494262,0.492609,285163.659000,9.652444,5,0,0,5,0,5,10,10.0,25.0,279636.0,280000.0,140000.0
2,2022-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,636730.75,258661.77,378068.98,0.593766,884323.87,0.590115,0.586285,0.573676,0.551173,248252.414387,14.248786,5,5,5,15,15,15,10,10.0,35.0,215927.0,220000.0,110000.0
3,2022-09-30,623e39dfbcd2bd3c1fb82f46,623e39dfbcd2bd3c1fb82f2c,218596.06,97528.40,121067.66,0.553842,261667.02,0.553887,0.548914,0.483571,0.483571,178099.270333,5.876880,5,-2,-2,1,0,1,0,10.0,11.0,178099.0,180000.0,90000.0
4,2022-09-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,366812.95,167578.21,199234.74,0.543151,503396.02,0.545345,0.542059,0.534984,0.523590,239122.137167,8.420735,5,5,5,15,0,15,10,10.0,35.0,227861.0,230000.0,115000.0
5,2022-09-30,5faf10dda4594c08c922b618,5faf10dda4594c08c922b600,365122.15,178329.67,186792.48,0.511589,535724.43,0.515596,0.504899,0.498357,0.489332,272400.058324,7.866730,5,0,-2,3,0,3,10,10.0,23.0,260210.0,260000.0,130000.0
6,2022-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,1043956.68,492541.26,551415.42,0.528198,1505519.11,0.529294,0.523984,0.514009,0.500611,363158.299333,16.582511,5,5,5,15,15,15,10,10.0,35.0,360785.0,360000.0,180000.0
7,2022-09-30,610daf318c9aff4915efe183,610daf318c9aff4915efe16a,160258.92,74694.29,85564.63,0.533915,219712.83,0.538467,0.534984,0.339953,0.404484,102272.556000,8.593227,5,-2,-2,1,0,1,10,10.0,21.0,99838.0,100000.0,50000.0
8,2022-09-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,508372.13,258151.46,250220.67,0.492200,746144.68,0.496285,0.500204,0.502461,0.495048,268164.234762,11.129667,5,5,0,10,15,15,10,10.0,35.0,260825.0,260000.0,130000.0
9,2022-09-30,609d85d3b6a049435d4faf02,609d85d3b6a049435d4faee9,345868.26,175702.28,170165.98,0.491997,437506.65,0.466779,0.455398,0.428353,0.432272,281156.240333,6.224392,-2,-2,-2,-6,0,-6,10,10.0,14.0,250532.0,250000.0,125000.0


In [65]:
final[['year_month', 'shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs', 'profit_pre_tax_post_discount',
       'margin_pre_tax_post_discount', 'sum_cogs_3m',
       'avg_margin_2m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all', 'inventory_fresh_and_stale',
       'inventory_turnover', 'gm_3m_score', 'gm_6m_score', 'gm_all_score', 'total_gm_perc_score','gm_dollar_score',
       'total_gm_score','turnover_score', 'rev_change_score',  'total_score', 'credit_limit_raw', 'potential_credit_limit', 'initial_credit_limit']].to_csv('blaze_ma_oct_all.csv')


# Proposal Analysis

In [66]:
final['total_score'].describe()

count    10.000000
mean     26.900000
std       9.433392
min      11.000000
25%      21.500000
50%      30.000000
75%      35.000000
max      35.000000
Name: total_score, dtype: float64

In [67]:
pass_list = final[final['total_score'] >= 24]

In [70]:
final_pass = pass_list[(pass_list['profit_pre_tax_post_discount'] >= 25000) & (pass_list['potential_credit_limit'] >= 30000)]
final_pass = final_pass.reset_index(drop = True)

In [71]:
final_pass

,year_month,shopid,companyid,rev_pre_tax_post_discount,lag_rev_pre_tax_post_discount,cogs,profit_pre_tax_post_discount,margin_pre_tax_post_discount,month_available,sum_cogs_3m,avg_margin_2m,avg_margin_3m,avg_margin_6m,avg_margin_all,rev_change,rev_change_bm,rev_change_weight,rev_change_var,rev_change_points,rev_change_total,inventory_fresh,inventory_stale,inventory_fresh_and_stale,inventory_turnover,gm_3m_score,gm_6m_score,gm_all_score,turnover_score,rev_change_score,gm_dollar_score,total_gm_perc_score,total_gm_score,total_score,credit_limit_raw,potential_credit_limit,initial_credit_limit
0,2022-09-30,607701db4683fe08ef9f2b44,607701db4683fe08ef9f2b2b,462616.26,448020.93,218091.65,244524.61,0.528569,10,645830.08,0.526690,0.525532,0.521119,0.527231,0.032577,NaN,NaN,NaN,NaN,NaN,296943.076000,47574.284000,344517.360000,7.498375,5,5,5,10,10.0,0,15,15,35.0,296943.0,300000.0,150000.0
1,2022-09-30,60b913136fc90078bb683ee1,602c09ee9e113008ed666e4a,500332.58,447181.51,250567.88,249764.70,0.499197,12,688131.56,0.503090,0.503835,0.494262,0.492609,0.118858,NaN,NaN,NaN,NaN,NaN,279636.169667,5527.489333,285163.659000,9.652444,5,0,0,10,10.0,0,5,5,25.0,279636.0,280000.0,140000.0
2,2022-09-30,5e7e9d34f795d008e7136fe1,5e750658d3980a08d54ddafa,636730.75,709714.60,258661.77,378068.98,0.593766,14,884323.87,0.590115,0.586285,0.573676,0.551173,-0.102835,NaN,NaN,NaN,NaN,NaN,215927.094387,32325.320000,248252.414387,14.248786,5,5,5,10,10.0,15,15,15,35.0,215927.0,220000.0,110000.0
3,2022-09-30,5fdce3b36c42c608cc9c8b8b,5fdce3b36c42c608cc9c8b73,366812.95,366086.20,167578.21,199234.74,0.543151,14,503396.02,0.545345,0.542059,0.534984,0.523590,0.001985,NaN,NaN,NaN,NaN,NaN,227860.525167,11261.612000,239122.137167,8.420735,5,5,5,10,10.0,0,15,15,35.0,227861.0,230000.0,115000.0
4,2022-09-30,5fb5656cd5322c09020a0027,5faf10dda4594c08c922b600,1043956.68,1075289.27,492541.26,551415.42,0.528198,14,1505519.11,0.529294,0.523984,0.514009,0.500611,-0.029139,NaN,NaN,NaN,NaN,NaN,360785.283333,2373.016000,363158.299333,16.582511,5,5,5,10,10.0,15,15,15,35.0,360785.0,360000.0,180000.0
5,2022-09-30,602c09ee9e113008ed666e63,602c09ee9e113008ed666e4a,508372.13,490704.93,258151.46,250220.67,0.492200,14,746144.68,0.496285,0.500204,0.502461,0.495048,0.036004,NaN,NaN,NaN,NaN,NaN,260825.138095,7339.096667,268164.234762,11.129667,5,5,0,10,10.0,15,10,15,35.0,260825.0,260000.0,130000.0


In [72]:
def calculate_interest_rate(score, full_score):
    score_ratio = score / full_score
    placeholder = (1 + 0.5 * (1 - score_ratio)) * 0.015
    rate = placeholder * 12
    return round(placeholder, 4), round(rate, 4)

In [73]:
final_pass['annual_rate'] = [calculate_interest_rate(final_pass['total_score'][i],35)[1] for i in range(len(final_pass))]
final_pass['monthly_rate'] = [calculate_interest_rate(final_pass['total_score'][i],35)[0] for i in range(len(final_pass))]



In [75]:
final_pass[['shopid','annual_rate','monthly_rate']]

,shopid,annual_rate,monthly_rate
0,607701db4683fe08ef9f2b44,0.1800,0.0150
1,60b913136fc90078bb683ee1,0.2057,0.0171
2,5e7e9d34f795d008e7136fe1,0.1800,0.0150
3,5fdce3b36c42c608cc9c8b8b,0.1800,0.0150
4,5fb5656cd5322c09020a0027,0.1800,0.0150
5,602c09ee9e113008ed666e63,0.1800,0.0150


In [ ]:
final_pass['credit_limit_final'] = [min(max(final_pass['credit_limit'][i],50000),250000) for i in range(len(final_pass))]

In [ ]:
final_pass[['year_month', 'shopid', 'companyid', 'rev_pre_tax_post_discount', 'cogs', 'profit_pre_tax_post_discount',
       'margin_pre_tax_post_discount', 'sum_cogs_3m',
       'avg_margin_2m', 'avg_margin_3m', 'avg_margin_6m', 'avg_margin_all', 'inventory_fresh_and_stale',
       'inventory_turnover', 'gm_3m_score', 'gm_6m_score', 'gm_all_score', 'total_gm_perc_score','gm_dollar_score',
       'total_gm_score','turnover_score', 'rev_change_score',  'total_score', 'credit_limit','annual_rate','monthly_rate']].to_csv('blaze_ma_sep_approval.csv')




In [ ]:
final

In [ ]:
final_pass

# ~ END ~

In [ ]:
# ['5e7e9d34f795d008e7136fe1', '602c09ee9e113008ed666e63',
#        '5fb5656cd5322c09020a0027', '5fdce3b36c42c608cc9c8b8b',
#        '607701db4683fe08ef9f2b44']

In [ ]:
final_pass

In [ ]:
final_pass['credit_limit'].describe()

In [ ]:
final_pass['credit_limit'].sum()

In [ ]:
final_pass['credit_limit_final'].describe()

In [ ]:
final_pass['credit_limit_final'].sum()

In [ ]:
final_pass.shape[0]/335

In [ ]:
final_pass.shape[0] / blaze_gm_data.shape[0]

In [ ]:
blaze_gm_data['shopid'].nunique()

In [ ]:
final_pass['rev_pre_tax_post_discount'].describe()

In [ ]:
final_pass

In [ ]:
#final_pass.to_csv('final_pass_0421_1.csv')

# wow rev change

In [ ]:
#you can also query like this
wow = pd.read_sql_query("""
with base as
(
   select
      *,
      --last_day(week_start) year_month,
      retail_value / quantity_sold per_unit_price,
      cogs / quantity_sold per_unit_cost,
      sales.retail_value - sales.total_discount + sales.total_tax sales_post_tax_post_discount,
      sales.retail_value - sales.total_discount sales_pre_tax_post_discount,
      sales.retail_value sales_pre_tax_pre_discount,
   from
      blaze_data.sales sales
      -- EXCLUDE SAMPLES,DISPLAY,PROMO and test
   where
      lower(product_category) not like '%sample%'
      and lower(product_category) not like '%display%'
      and lower(product_category) not like '%promo%'
     and lower(product_category) not like '%test%'
)


,
monthly_location_rev_and_cogs as
(
   select
      week_start,
      week_end,
      shopid,
      companyid,
      sum(sales_post_tax_post_discount) rev_post_tax_post_discount,
      sum(sales_pre_tax_post_discount) rev_pre_tax_post_discount,
      sum(sales_pre_tax_pre_discount) rev_pre_tax_pre_discount,
      sum(cogs) cogs
   from
      base
      -- EXCLUDE low COGS items
   where
      cogs > 1
      -- EXCLUDE high cost, high price items
      and per_unit_price <= 500
      and per_unit_cost <= 500
   group by
      3,
      4,
      1,
      2
   order by
      3,
      4,
      1,
      2
)
,
monthly_location_rev_and_cogs_and_profit as
(
   select
      week_start,
      week_end,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      rev_post_tax_post_discount - cogs profit_post_tax_post_discount,
      rev_pre_tax_post_discount - cogs profit_pre_tax_post_discount,
      rev_pre_tax_pre_discount - cogs profit_pre_tax_pre_discount,
   from
      monthly_location_rev_and_cogs
)
,
monthly_location_rev_and_cogs_and_profit_and_margin as
(
   select
      week_start,
      week_end,
      shopid,
      companyid,
      -- rev and cogs
      rev_post_tax_post_discount,
      rev_pre_tax_post_discount,
      rev_pre_tax_pre_discount,
      cogs,
      -- profit
      profit_post_tax_post_discount,
      profit_pre_tax_post_discount,
      profit_pre_tax_pre_discount,
      -- margin
      profit_post_tax_post_discount / nullif(rev_post_tax_post_discount, 0) margin_post_tax_post_discount,
      profit_pre_tax_post_discount / nullif(rev_pre_tax_post_discount, 0) margin_pre_tax_post_discount,
      profit_pre_tax_pre_discount / nullif(rev_pre_tax_pre_discount, 0) margin_pre_tax_pre_discount
   from
      monthly_location_rev_and_cogs_and_profit
)
,
monthly_gm_analysis as
(
   select
      week_start,
      week_end,
      shopid,
      companyid,
      -- CHOOSE TO USE PRE TAX POST DISCOUNT
      rev_pre_tax_post_discount,
      lag(rev_pre_tax_post_discount) over (partition by shopid order by week_start) lag_rev_pre_tax_post_discount,
      cogs,
      profit_pre_tax_post_discount,
      margin_pre_tax_post_discount,
      count(distinct week_start) over (partition by shopid ) month_available
    --  sum(cogs) over (partition by shopid order by week_start rows between 2 preceding and current row ) sum_cogs_3m,
     -- avg(margin_pre_tax_post_discount) over (partition by shopid order by week_start rows between 2 preceding and current row ) avg_margin_3m,
     -- avg(margin_pre_tax_post_discount) over (partition by shopid order by week_start rows between 5 preceding and current row ) avg_margin_6m,
      --avg(margin_pre_tax_post_discount) over (partition by shopid order by week_start ) avg_margin_all
   from
      monthly_location_rev_and_cogs_and_profit_and_margin
)
select
   *,
   (
      rev_pre_tax_post_discount - lag_rev_pre_tax_post_discount
   )
   / nullif(lag_rev_pre_tax_post_discount, 0) rev_change
from
   monthly_gm_analysis
where shopid not in ("5e50449d6c0a910843356fa3",
"5e666af1aa809b085a98a646",
"56cf846ee38179985229e59e",
"5e6d0948b987950831b5092c",
"5ab806d6c182bb072e0b0e01",
"617b7da0e5cb65322b32781a",
"56cf84e4e38179985229e59f",
"5ee7c6a938fa9a08c8b7939c",
"5eebecd661ea0b08cdbcc386",
"56cf855ce38179985229e5a0",
"5ef12ad7bb5e7a08c5b3f4f3",
"5f1108355ae12008f4fef76e",
"5f123a5d12497908d2888dde",
"5f2309d579373608c93e6742",
"5f341ed02fdd5608e0821b45",
"5f8870200260fe08fe708db5",
"5f8870200260fe08fe708db6",
"5fac6a0b0643cc08efd1d327",
"56cf855ce38179985229e5a1",
"5d238c1955be2e07f5ee66e5",
"57b35a5d920f401e40146301",
"603d1a9eb9d7dc08cca27203",
"603d1a9eb9d7dc08cca27204",
"6046544f61e17808cff96fdb",
"6046544f61e17808cff96fdc",
"57b35aa8920f401e40146334",
"57b6b145920f4045cbe5fe81",
"6059219abe32aa08fa3a11dc",
"57b6b36a920f4046337f2618",
"58a7433f819d9e6ebd427617",
"60c0ffd6331cab296f02f49a",
"610c0a0d381ba0747c981ce9",
"611c0c98127cdd740cff94db",
"5a43f72b819d9e3ef9373675",
"611c0cc403d2e97dc2c159b0",
"5e6d0ac5b987950831b513e3",
"5a4563f1819d9e4ae6207e06",
"611c0ce7649819050ed4906b",
"611c0d02f572ec7882d64eea",
"611c0d26d5f1e86542fe3b8b",
"61672305b5e382719c7e4c2a",
"616dee0e4497e822dae13b21",
"616e5b13dd10fe4d3f34fd05",
"616e5b13dd10fe4d3f34fd06",
"619834e921c8017e23135674",
"5ab2d1a5819d9e21137cec3c",
"61bb746b82fddc50e058d711",
"61c3be757511d81170a29d51",
"61d49080eec1b51a3e33dc39",
"61d8912bdd71d40c2b68c5b3",
"61ba4efb9536472be6a8baa9",
"61d895fb3efbfb6da5210ae3",
"61d8aaf527205210191d7a1e",
"61e72d51073c4a79cfeccb62",
"61fdab9eae59d05c49422c3a",
"6202bd1c15425361f688aabf",
"6202bd1c15425361f688aac0",
"5b9f8e1db93307080d35b85f",
"5c2c250eabe12808989ac2f7",
"58be5603c182bb1f9bcd2ad5",
"5c8be64665a6a40808118621",
"606761051436a608e166db19",
"5ce2eb562d4dcf07fb7d0c5f",
"6176fa9eda707b1faba5f717",
"5b517867485da70830fcc04e",
"5b9f841a17b6e00812fe71f3",
"5da023586d8ce1082b9c91bd",
"5bbce3db420c21083300e410",
"5d238c1955be2e07f5ee66e4",
"5b209459062bd807b142a1f6",
"5cb1a5839fbb5f0818f6de55",
"5cda1adb183d6a0801347027",
"5be3903b79015e083194aa70",
"5d1ba174cdbd6d0803c99cfd",
"5a8a6eb8819d9e78641ab6ec",
"5db863825a04a2083b464083",
"5d8a7fb3a3274308311743d5",
"5d8a7fb3a3274308311743d6",
"5d9e425d52d15b0824a8a5d5",
"5dd6da52ce4015084c72e8db",
"5e07f51837cdd7082662f3c6",
"5e4d849981e3cd08326b6c14",
"5dfd2308fa462a086a8bfab8",
"5e0ee2710a7b7008346b06b4",
"5e4d849981e3cd08326b6c15",
"5dfd2308fa462a086a8bfab9",
"5e6d0adc12f2e208728f9369",
"5e6d0948b987950831b5092b",
"5ed589d96c0bf408dcc92d97",
"5eebecd761ea0b08cdbcc387",
"5ef12ad7bb5e7a08c5b3f4f2",
"5f63afd430bd7208ce1ab304",
"5f63afd430bd7208ce1ab305",
"5bf5ce54062bd807e436b473",
"5fac691a715f3808cd0aa82d",
"5d9d7fc27e2e46081e7e69f3",
"6034287d52503608c0b3ca48",
"5dfd40ea521ea90860c91848",
"6034287d52503608c0b3ca49",
"60636227ec491108d1c970d5",
"5a9a2274c182bb4ec00a72ca",
"60aea3bb1940943cecec2dbf",
"60a2f1610f75543c8e7dde50",
"5eb98f2d649a6508f87a6436",
"5e62a68447d90a086b8231c0",
"5e62a68447d90a086b8231c1",
"615b634cac08aa0a794158c4",
"60a2db8d0f75543c8e7d513b",
"60a2db8d0f75543c8e7d513c",
"60a4051e67406e7edad8ab88",
"60c0ff66082dcc61e42ad4a6",
"60c0ff66082dcc61e42ad4a7",
"6046c57057abe808d1f260fd",
"60636227ec491108d1c970d4",
"610abcaec3d09859f9b630d1",
"606cfaadba253e08c4189a63",
"607a321441cfa308f6e65b58",
"6197e31b21c8017e2310c10e",
"611d457db7c6f80914292394",
"610c0a0d381ba0747c981cea",
"610abcaec3d09859f9b630d2",
"611c0cb3a29e0f598f108a60",
"5fff453a81a63608d3a1a5b7",
"615237cc0937e52684b6c134",
"620b59c97852ca7a7dcc6f05",
"6196aa4deb62c87eba4014ad",
"619816781f88a1280f8a0c28",
"61d7b217a42c2f124d9b76b1",
"61cf4b452bc6657512b92d2a",
"61ebb060b04dc7510465f774",
"61eedaa0b882812f2f46a23d",
"61eedade6716dd2a2e0cfce5",
"5e6d0ae6e4a25a0834047467",
"5dc3315fe0faca07c6dfe849",
"5f7e17594445b008d2f63ebd",
"5f7f63053dbcad08cc9edbfb",
"5fff453a81a63608d3a1a5b8",
"603d5425a9ca7208f886d156",
"60e48dbbd5596d7c8a1d24dc",
"611c0d335901162ff90ef4dd",
"613f771aa96c863c00c1517d",
"6153b70fd343dc6230c4b67d",
"61699cdf032e1c1f920ec70f",
"61699cdf032e1c1f920ec710",
"61a7b4c6c8c7d85ea1846f14",
"61c390cfb863cb3baddb70f4",
"61d7b252ae999342ac05499e",
"61e6f857c5da34762f1c1fb5",
"61ebb00a335a531542342a32",
"620b59c97852ca7a7dcc6f04",
"6226640755810d3311e8ecd0")
""",engine)

In [ ]:
wow

In [ ]:
wow_bm = wow.groupby(['week_start','week_end'])['rev_change'].describe().reset_index()
wow_bm.index = pd.to_datetime(wow_bm['week_start'])
wow_bm

In [ ]:
wow_bm[['50%']].plot(kind= 'bar',figsize = (10,8))